In [1]:
import requests
from datetime import datetime
import json
import time
import pandas as pd
from time import mktime

pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
ls

Analysing all the sectors.ipynb*
Outliers & Standard Deviation Calculations.ipynb
TickByTickMispricingAnalysis.ipynb
XLK-holdings20200220.csv
XLK-holdings20200221.csv
XLK-holdings20200224.csv
XLK-holdings20200225.csv
XLK-holdings20200226.csv
XLK-holdings20200228.csv


In [3]:
class PolgonData(object):

    def __init__(self):
        self.params=params = (('apiKey', 'M_PKVL_rqHZI7VM9ZYO_hwPiConz5rIklx893F'),)

    def PolygonLastTrades(self,symbol):
        # Make use of Tickers
        requesturl='https://api.polygon.io/v1/last/stocks/'+symbol
        response = requests.get(requesturl, params=self.params)
        return json.loads(response.text)

    def PolygonHistoricQuotes(self, date=None, symbol=None,startTS=None,endTS=None,limitresult=10):
        if startTS:
            # For Getting Paginated Request
            requesturl='https://api.polygon.io/v2/ticks/stocks/nbbo/'+symbol+'/'+date+'?timestamp='+startTS+'&timestampLimit='+endTS+'&limit='+limitresult
            print("Paginated Request For = " + symbol)
        else:
            requesturl='https://api.polygon.io/v2/ticks/stocks/nbbo/'+symbol+'/'+date+'?timestampLimit='+endTS+'&limit='+limitresult
            print("First Request For = " + symbol)
        print(requesturl)
        response = requests.get(requesturl, params=self.params)
        return json.loads(response.text)

    def PolygonHistoricTrades(self, date=None, symbol=None,startTS=None,endTS=None,limitresult=10):
        if startTS:
            # For Getting Paginated Request
            requesturl='https://api.polygon.io/v2/ticks/stocks/trades/'+symbol+'/'+date+'?timestamp='+startTS+'&timestampLimit='+endTS+'&limit='+limitresult
            print("Paginated Request For = " + symbol)
        else:
            requesturl='https://api.polygon.io/v2/ticks/stocks/trades/'+symbol+'/'+date+'?timestampLimit='+endTS+'&limit='+limitresult
            print("First Request For = " + symbol)
        print(requesturl)
        response = requests.get(requesturl, params=self.params)
        return json.loads(response.text)

    def PolygonDailyOpenClose(self,date=None, symbol=None):
        requesturl='https://api.polygon.io/v1/open-close/'+symbol+'/'+date
        response = requests.get(requesturl, params=self.params)
        return json.loads(response.text)
    
    def PolygonAggregdateData(self):
        # Make use of Tickers, Date and Limit
        requesturl='https://api.polygon.io/v2/aggs/ticker/AAPL/range/1/minute/2020-02-14/2020-02-15'
        response = requests.get(requesturl, params=self.params)
        return json.loads(response.text)

In [4]:
from datetime import datetime

class DateTimeManipulation(object):
    
    def __init__(self,date=None):
        self.date=date

    # Returns timestamp with milliseconds
    def unix_time_millis(self,dt):
        epoch = datetime.utcfromtimestamp(0)
        tsDate=(dt - epoch).total_seconds() * 1000.0
        tsDate=str(int(tsDate))+'000000'
        return tsDate

    def stringTimeToDatetime(self,date=None,time=None):
        marketOpenTSStr = date +' '+ time
        return datetime.strptime(marketOpenTSStr,'%Y-%m-%d %H:%M:%S')
    
    def convertStringDateToTS(self,starttime='9:30:00',endtime='17:00:00'):
        marketOpenTSStr = self.date +' '+ starttime
        marketCloseTSStr = self.date +' ' + endtime
        
        marketTimeStamps={}
        marketTimeStamps['marketOpenTS']=self.unix_time_millis(datetime.strptime(marketOpenTSStr,'%Y-%m-%d %H:%M:%S'))
        marketTimeStamps['marketCloseTS']=self.unix_time_millis(datetime.strptime(marketCloseTSStr,'%Y-%m-%d %H:%M:%S'))
        return marketTimeStamps
    
    def getHumanTime(self,ts,getMilliSecondsAlso=False):
        try:
            s, ms = divmod(ts, 1000000000)
            if getMilliSecondsAlso:
                return datetime(*time.gmtime(s)[:6]),ms
            else:
                return datetime(*time.gmtime(s)[:6])
            #print('{}.{:03d}'.format(time.strftime('%Y-%m-%d %H:%M:%S',  time.gmtime(s)), ms))
        except AttributeError:
            print("Attribute Error Occured")
            print(ts)
            print(s)
            print(ms)
            

In [5]:
# Taking in ETF List
holdings=pd.read_csv("XLK-holdings20200228.csv")

holdings['Weighting']=holdings['Weighting'].apply(lambda x:x.replace('%',''))
holdings['Weighting']=holdings['Weighting'].astype(float)
holdings['Weighting']=holdings['Weighting']/100
weights=dict(zip(holdings.Symbol,holdings.Weighting))

cashvalue=holdings[holdings['Symbol']=='CASH'].get('Weighting').item()*28583351000

symbols=list(holdings['Symbol'].values)+['XLK']
symbols.remove('CASH')

# Process the ticker data

XLK: Technology Select Sector SPDR Fund<br>
Inception Date: 1998-12-16<br>
Fund Holdings as of: 2020-02-28<br>
"Total Assets Under Management (in thousands):	24922689"<br>
Shares Outstanding: 281806000<br>
Expense Ratio: 0.13%<br>
Tracks This Index: Technology Select Sector Index<br>
ETFdb.com Category: Technology Equities<br>
Issuer: State Street SPDR<br>


In [13]:
# Create an object of date when we need and time between which we need data
previousdate='2020-02-28'
date='2020-03-02'
starttime='9:30:00'
endtime='17:00:00'
endtimeLoop='16:00:00'

class GetETFFrame(object):
    
    def __init__(self):
        self.date=date
        self.starttime=starttime
        self.endtime=endtime
        self.endtimeLoop=endtimeLoop
        self.extractDataTillTime = DateTimeManipulation().stringTimeToDatetime(date=self.date,time=self.endtimeLoop)
        self.datetimeObj=DateTimeManipulation(self.date)
        self.marketTimeStamps=self.datetimeObj.convertStringDateToTS(starttime=self.starttime,endtime=self.endtime)

    def getDataFromPolygon(self,symbol,methodPassed):
        tickHistData={}
        # First Request
        data=methodPassed(date=self.date,symbol=symbol,endTS=self.marketTimeStamps['marketCloseTS'],limitresult=str(50000))
        # Last timestamp from data received
        lastUnixTimeStamp = data['results'][-1]['t']
        # Covert UNIX timestamp to human timestamp
        lastHumanTimeStamp = self.datetimeObj.getHumanTime(lastUnixTimeStamp)
        # Get timestamp for date +  '18:00:00' hrs - Make use of pagination
        # Paginated Request if the data from above doesn't reach 5 pm time
        while lastHumanTimeStamp < self.extractDataTillTime:
            print(self.datetimeObj.getHumanTime(data['results'][-1]['t']))
            data2=methodPassed(date=self.date,symbol=symbol,startTS=str(lastUnixTimeStamp),endTS=self.marketTimeStamps['marketCloseTS'],limitresult=str(50000))
            # Last timestamp from data received
            lastUnixTimeStamp = data2['results'][-1]['t']
            # Covert UNIX timestamp to human timestamp
            lastHumanTimeStamp = self.datetimeObj.getHumanTime(lastUnixTimeStamp)
            # Get timestamp for date +  '18:00:00' hrs - Make use of pagination
            data['results']=data['results'] + data2['results']
        tickHistData[symbol] = data
        return tickHistData
    
import concurrent.futures    
executor = concurrent.futures.ProcessPoolExecutor(20)
#########**************#########
# Needs Threading #
#########**************#########
tickHistDataQuotes={}
tickHistDataTrade={}
ob=GetETFFrame()
for symbol in symbols:
    tickHistDataQuotes[symbol]=ob.getDataFromPolygon(symbol,PolgonData().PolygonHistoricQuotes)
    tickHistDataTrade[symbol]=ob.getDataFromPolygon(symbol,PolgonData().PolygonHistoricTrades)

First Request For = MSFT
https://api.polygon.io/v2/ticks/stocks/nbbo/MSFT/2020-03-02?timestampLimit=1583168400000000000&limit=50000
2020-03-02 14:46:37
Paginated Request For = MSFT
https://api.polygon.io/v2/ticks/stocks/nbbo/MSFT/2020-03-02?timestamp=1583160397235025568&timestampLimit=1583168400000000000&limit=50000
2020-03-02 15:06:27
Paginated Request For = MSFT
https://api.polygon.io/v2/ticks/stocks/nbbo/MSFT/2020-03-02?timestamp=1583161587695160507&timestampLimit=1583168400000000000&limit=50000
2020-03-02 15:34:15
Paginated Request For = MSFT
https://api.polygon.io/v2/ticks/stocks/nbbo/MSFT/2020-03-02?timestamp=1583163255270764659&timestampLimit=1583168400000000000&limit=50000
First Request For = MSFT
https://api.polygon.io/v2/ticks/stocks/trades/MSFT/2020-03-02?timestampLimit=1583168400000000000&limit=50000
2020-03-02 14:38:10
Paginated Request For = MSFT
https://api.polygon.io/v2/ticks/stocks/trades/MSFT/2020-03-02?timestamp=1583159890437260875&timestampLimit=1583168400000000000&

First Request For = CRM
https://api.polygon.io/v2/ticks/stocks/nbbo/CRM/2020-03-02?timestampLimit=1583168400000000000&limit=50000
First Request For = CRM
https://api.polygon.io/v2/ticks/stocks/trades/CRM/2020-03-02?timestampLimit=1583168400000000000&limit=50000
First Request For = PYPL
https://api.polygon.io/v2/ticks/stocks/nbbo/PYPL/2020-03-02?timestampLimit=1583168400000000000&limit=50000
2020-03-02 15:21:00
Paginated Request For = PYPL
https://api.polygon.io/v2/ticks/stocks/nbbo/PYPL/2020-03-02?timestamp=1583162460896687226&timestampLimit=1583168400000000000&limit=50000
First Request For = PYPL
https://api.polygon.io/v2/ticks/stocks/trades/PYPL/2020-03-02?timestampLimit=1583168400000000000&limit=50000
First Request For = IBM
https://api.polygon.io/v2/ticks/stocks/nbbo/IBM/2020-03-02?timestampLimit=1583168400000000000&limit=50000
2020-03-02 15:33:17
Paginated Request For = IBM
https://api.polygon.io/v2/ticks/stocks/nbbo/IBM/2020-03-02?timestamp=1583163197279550000&timestampLimit=1583

First Request For = ADSK
https://api.polygon.io/v2/ticks/stocks/nbbo/ADSK/2020-03-02?timestampLimit=1583168400000000000&limit=50000
First Request For = ADSK
https://api.polygon.io/v2/ticks/stocks/trades/ADSK/2020-03-02?timestampLimit=1583168400000000000&limit=50000
First Request For = ADI
https://api.polygon.io/v2/ticks/stocks/nbbo/ADI/2020-03-02?timestampLimit=1583168400000000000&limit=50000
First Request For = ADI
https://api.polygon.io/v2/ticks/stocks/trades/ADI/2020-03-02?timestampLimit=1583168400000000000&limit=50000
First Request For = CTSH
https://api.polygon.io/v2/ticks/stocks/nbbo/CTSH/2020-03-02?timestampLimit=1583168400000000000&limit=50000
First Request For = CTSH
https://api.polygon.io/v2/ticks/stocks/trades/CTSH/2020-03-02?timestampLimit=1583168400000000000&limit=50000
First Request For = HPQ
https://api.polygon.io/v2/ticks/stocks/nbbo/HPQ/2020-03-02?timestampLimit=1583168400000000000&limit=50000
2020-03-02 15:13:08
Paginated Request For = HPQ
https://api.polygon.io/v2/ti

First Request For = CTXS
https://api.polygon.io/v2/ticks/stocks/trades/CTXS/2020-03-02?timestampLimit=1583168400000000000&limit=50000
First Request For = BR
https://api.polygon.io/v2/ticks/stocks/nbbo/BR/2020-03-02?timestampLimit=1583168400000000000&limit=50000
First Request For = BR
https://api.polygon.io/v2/ticks/stocks/trades/BR/2020-03-02?timestampLimit=1583168400000000000&limit=50000
First Request For = QRVO
https://api.polygon.io/v2/ticks/stocks/nbbo/QRVO/2020-03-02?timestampLimit=1583168400000000000&limit=50000
First Request For = QRVO
https://api.polygon.io/v2/ticks/stocks/trades/QRVO/2020-03-02?timestampLimit=1583168400000000000&limit=50000
First Request For = JKHY
https://api.polygon.io/v2/ticks/stocks/nbbo/JKHY/2020-03-02?timestampLimit=1583168400000000000&limit=50000
First Request For = JKHY
https://api.polygon.io/v2/ticks/stocks/trades/JKHY/2020-03-02?timestampLimit=1583168400000000000&limit=50000
First Request For = IT
https://api.polygon.io/v2/ticks/stocks/nbbo/IT/2020-0

In [14]:
previoousdaylastprieforNaFilling={}
samedaylastprieforNaFilling={}
lastprieforNaFilling={}
ob=PolgonData()
for symbol in symbols:
    d2=ob.PolygonDailyOpenClose(date=date,symbol=symbol)
    lastprieforNaFilling[symbol]=d2['open']

In [19]:
tickerO='AAPL'

print(lastprieforNaFilling[tickerO])

283.15


# Meaning of Each Columns
## Quotes Data Mapping 


 'i': {'name': 'indicators', 'type': '[]int' <br>
 's': {'name': 'bid_size', 'type': 'int'}, <br>
 'x': {'name': 'bid_exchange', 'type': 'int'}, <br>
 'P': {'name': 'ask_price', 'type': 'float64'}, <br>
 'S': {'name': 'ask_size', 'type': 'int'}, <br>
 't': {'name': 'sip_timestamp', 'type': 'int64'}, <br>
 'f': {'name': 'trf_timestamp', 'type': 'int64'}, <br>
 'c': {'name': 'conditions', 'type': '[]int'}, <br>
 'z': {'name': 'tape', 'type': 'int'}, <br>
 'X': {'name': 'ask_exchange', 'type': 'int'}, <br>
 'y': {'name': 'participant_timestamp', 'type': 'int64'}, <br>
 'q': {'name': 'sequence_number', 'type': 'int'}, <br>
 'p': {'name': 'bid_price', 'type': 'float64'} <br>
 
 
 ## Trade Data Mapping 


 'y': {'name': 'participant_timestamp', 'type': 'int64'} <br>
 'i': {'name': 'id', 'type': 'string'} <br>
 'e': {'name': 'correction', 'type': 'int'} <br>
 'x': {'name': 'exchange', 'type': 'int'} <br>
 'r': {'name': 'trf_id', 'type': 'int'} <br>
 'p': {'name': 'price', 'type': 'float64'} <br>
 'z': {'name': 'tape', 'type': 'int'} <br>
 't': {'name': 'sip_timestamp', 'type': 'int64'} <br>
 'f': {'name': 'trf_timestamp', 'type': 'int64'} <br>
 'q': {'name': 'sequence_number', 'type': 'int'} <br>
 'c': {'name': 'conditions', 'type': '[]int'} <br>
 'I': {'name': 'orig_id', 'type': 'string'} <br>
 's': {'name': 'size', 'type': 'int'} <br>

 
 ### Clean the Holdings data

In [20]:
def quotesDataCleaning(tickHistDataQuotes):
    finalDF=[]
    appendData=[]
    for key,value in tickHistDataQuotes.items():
        df=pd.DataFrame.from_dict(value[key]['results'])
        df['Symbol']=key
        appendData.append(df)
    finalDF= pd.concat(appendData)
    finalDF=finalDF[['Symbol','P','S','p','s','t','x','X']]
    finalDF=finalDF[finalDF['S']!=0]
    finalDF=finalDF[finalDF['s']!=0]
    return finalDF
    
def tradeDataCleaning(tickHistDataTrade):
    finalDF=[]
    appendData=[]
    for key,value in tickHistDataTrade.items():
        df=pd.DataFrame.from_dict(value[key]['results'])
        df['Symbol']=key
        appendData.append(df)
    finalDF= pd.concat(appendData)
    finalDF=finalDF[['Symbol','p','s','t','x']]
    finalDF=finalDF[finalDF['s']!=0]
    return finalDF
    

In [21]:
# Convert Response data to Dataframes for quotes and trade data
tradeData=tradeDataCleaning(tickHistDataTrade)
quotesData=quotesDataCleaning(tickHistDataQuotes)

In [ ]:
# Deleting to save space on disk
del tickHistDataTrade
del tickHistDataQuotes

## Exhanges list

In [ ]:
exchanges=[{"id":0,"type":"TRF","market":"equities","mic":"TFF","name":"Multiple","tape":"-"},{"id":1,"type":"exchange","market":"equities","mic":"XASE","name":"NYSE American (AMEX)","tape":"A","code":"AMX"},{"id":2,"type":"exchange","market":"equities","mic":"XNAS","name":"NASDAQ OMX BX","tape":"B","code":"NSD"},{"id":3,"type":"TRF","market":"equities","name":"National Stock Exchange","mic":"XCIS","tape":"C"},{"id":4,"type":"TRF","market":"equities","mic":"FINR","name":"FINRA","tape":"D"},{"id":5,"type":"TRF","market":"equities","mic":"CQS","name":"Consolidated Quote System","tape":"E"},{"id":6,"type":"TRF","market":"equities","mic":"XISX","name":"International Securities Exchange","tape":"I","code":"IOE"},{"id":7,"type":"exchange","market":"equities","mic":"EDGA","name":"Cboe EDGA","tape":"J","code":"XCBO"},{"id":8,"type":"exchange","market":"equities","mic":"EDGX","name":"Cboe EDGX","tape":"K","code":"EDGX"},{"id":9,"type":"exchange","market":"equities","mic":"XCHI","name":"Chicago Stock Exchange, Inc","tape":"M"},{"id":10,"type":"exchange","market":"equities","mic":"XNYS","name":"New York Stock Exchange","tape":"N","code":"NYE"},{"id":11,"type":"exchange","market":"equities","mic":"ARCX","name":"NYSE Arca","tape":"P","code":"ARCA"},{"id":12,"type":"exchange","market":"equities","mic":"XNGS","name":"Nasdaq","tape":"T","code":"NSD"},{"id":13,"type":"TRF","market":"equities","mic":"CTS","name":"Consolidated Tape System","tape":"S"},{"id":14,"type":"TRF","market":"equities","mic":"OOTC","name":"OTC Bulletin Board","tape":"U"},{"id":141,"type":"TRF","market":"equities","mic":"XOTC","name":"OTC Bulletin Board","tape":"U"},{"id":142,"type":"TRF","market":"equities","mic":"PSGM","name":"OTC Bulletin Board","tape":"U","code":"GREY"},{"id":143,"type":"TRF","market":"equities","mic":"PINX","name":"OTC Bulletin Board","tape":"U","code":"OTO"},{"id":144,"type":"TRF","market":"equities","mic":"OTCB","name":"OTC Bulletin Board","tape":"U","code":"OTCQB"},{"id":145,"type":"TRF","market":"equities","mic":"OTCQ","name":"OTC Bulletin Board","tape":"U","code":"OTCQX"},{"id":15,"type":"exchange","market":"equities","mic":"IEXG","name":"IEX","tape":"V","code":"IEXG"},{"id":16,"type":"TRF","market":"equities","mic":"XCBO","name":"Chicago Board Options Exchange","tape":"W","code":"CBO"},{"id":17,"type":"exchange","market":"equities","mic":"PHLX","name":"Nasdaq PSX","tape":"X"},{"id":18,"type":"exchange","market":"equities","mic":"BATY","name":"Cboe BYX","tape":"Y","code":"BATS"},{"id":19,"type":"exchange","market":"equities","mic":"BATS","name":"Cboe BZX","tape":"Z","code":"BATS"},{"id":33,"type":"exchange","market":"equities","mic":"XBOS","name":"NASDAQ BX Options/ETF","tape":"B"},{"id":36,"type":"exchange","market":"index","name":"CME S&P Complete Indices","code":"SPIC"},{"id":37,"type":"exchange","market":"index","name":"Russell Tick Indices","code":"RUS"},{"id":38,"type":"exchange","market":"index","name":"CSMI Indices Exchange","code":"MDX"},{"id":39,"type":"exchange","market":"index","name":"CME S&P Base Indices","code":"SPIB"},{"id":40,"type":"exchange","market":"index","name":"Dow Jones Indexes","code":"DJI"},{"id":20,"type":"banking","market":"currencies","name":"Currency Banks 1"},{"id":44,"type":"banking","market":"currencies","name":"Currency Banks 2"},{"id":60,"type":"banking","market":"currencies","name":"Currency Banks 3"}]
pd.DataFrame.from_dict(exchanges)

### Perform Analysis

In [22]:
# Clean the DataFrame
#########**************#########
# Needs Threading in time conversion #
#########**************#########
tradeData['t']=tradeData['t'].apply(lambda x:DateTimeManipulation().getHumanTime(x,getMilliSecondsAlso=False))

#########**************#########
# Needs Threading in time conversion #
#########**************#########
quotesData['t']=quotesData['t'].apply(lambda x:DateTimeManipulation().getHumanTime(x,getMilliSecondsAlso=False))
quotesData['Spread']=quotesData['P']-quotesData['p']
quotesData['MidPrice']=(quotesData['P']+quotesData['p'])/2

In [23]:
print(tradeData.head(5))
print(quotesData.head(5))

  Symbol       p     s                   t   x
0   MSFT  166.15  3695 2020-03-02 09:00:00  11
1   MSFT  166.15   207 2020-03-02 09:00:00  11
2   MSFT  166.43    19 2020-03-02 09:00:00  11
3   MSFT  166.43     8 2020-03-02 09:00:00  11
4   MSFT  166.00     3 2020-03-02 09:00:00  11
  Symbol       P   S      p   s                   t   x   X  Spread  MidPrice
1   MSFT  166.43  21  165.0  10 2020-03-02 09:00:11  12  11    1.43   165.715
2   MSFT  166.42   1  165.0  10 2020-03-02 09:00:13  12  11    1.42   165.710
3   MSFT  166.41  21  165.0  10 2020-03-02 09:00:13  12  11    1.41   165.705
4   MSFT  166.40   1  165.0  10 2020-03-02 09:00:13  12  11    1.40   165.700
5   MSFT  166.39  21  165.0  10 2020-03-02 09:00:13  12  11    1.39   165.695


In [24]:
tradePrices=tradeData.groupby([tradeData['t'].dt.hour,tradeData['t'].dt.minute,tradeData['Symbol']])['p'].mean()
tradePricesDF=tradePrices.unstack(level=2)
tradePricesDF=tradePricesDF.fillna(method='ffill')
tradePricesDF=tradePricesDF.fillna(lastprieforNaFilling)

quotesSpreads=quotesData.groupby([quotesData['t'].dt.hour,quotesData['t'].dt.minute,quotesData['Symbol']])['Spread'].mean()
quotesSpreadDF=quotesSpreads.unstack(level=2)
quotesSpreadDF=quotesSpreadDF.fillna(0)

In [25]:
def getMeHourdata(getmeHourDataFor=None):
    etfticker='XLK'
    etfspread=quotesSpreadDF[etfticker]
    
    for name,group in tradePricesDF.groupby(level=0):
        if name==getmeHourDataFor:
            break

    etfprice=group[etfticker]
    del group[etfticker]
    
    group=group.pct_change().dropna()*100
    
    etfpricechange=etfprice.pct_change().dropna()*100
    etfpricechange=etfpricechange.unstack(level=1)
    
    netassetvaluereturn=group.assign(**weights).mul(group).sum(axis=1)
    netassetvaluereturn=netassetvaluereturn.unstack(level=1)

    ds=pd.concat([etfprice.unstack(level=1),etfpricechange,netassetvaluereturn],axis=0).T
    ds.columns=['ETF Price','ETF Change Price %','Net Asset Value Change%']

    ds['Arbitrage in $']=(ds['ETF Change Price %'] - ds['Net Asset Value Change%'])*ds['ETF Price']/100
    ds['ETF Trading Spread in $']=etfspread.unstack(level=1).loc[getmeHourDataFor]
    return ds



In [26]:
for i in range(9,21):
    print("Hour at="+str(i))
    res=getMeHourdata(i)
    res['Arbitrage in $']=abs(res['Arbitrage in $'])
    res['Flag']=0
    res.loc[(res['Arbitrage in $']>res['ETF Trading Spread in $']) & res['ETF Trading Spread in $']!=0,'Flag']=111
    print(res)

Hour at=9
    ETF Price  ETF Change Price %  Net Asset Value Change%  Arbitrage in $  ETF Trading Spread in $  Flag
t                                                                                                        
0      89.850                 NaN                      NaN             NaN                 0.000000     0
1      89.850            0.000000                -0.129204        0.116090                 0.000000     0
2      89.850            0.000000                 0.026181        0.023524                 0.000000     0
3      89.875            0.027824                 0.025768        0.001848                 0.000000     0
4      89.875            0.000000                 0.056230        0.050537                 0.000000     0
5      89.875            0.000000                 0.114173        0.102613                 0.000000     0
6      89.875            0.000000                 0.091774        0.082482                 0.000000     0
7      89.875            0.000000   

    ETF Price  ETF Change Price %  Net Asset Value Change%  Arbitrage in $  ETF Trading Spread in $  Flag
t                                                                                                        
0   87.749643                 NaN                      NaN             NaN                 0.489037     0
1   87.780000            0.034595                 0.039533        0.004334                 0.369937     0
2   87.750000           -0.034176                 0.053543        0.076974                 0.366794     0
3   87.780000            0.034188                 0.025139        0.007943                 0.306792     0
4   87.836667            0.064555                -0.097211        0.142090                 0.122229   111
5   87.808571           -0.031986                -0.013316        0.016394                 0.248667     0
6   87.808571            0.000000                -0.000540        0.000474                 0.281923     0
7   87.885000            0.087040             

    ETF Price  ETF Change Price %  Net Asset Value Change%  Arbitrage in $  ETF Trading Spread in $  Flag
t                                                                                                        
0   89.290000                 NaN                      NaN             NaN                 0.236303     0
1   89.295000            0.005600                 0.034939        0.026199                 0.306646     0
2   89.295000            0.000000                 0.068527        0.061191                 0.282468     0
3   89.210000           -0.095190                 0.065795        0.143615                 0.167988     0
4   89.192000           -0.020177                -0.041785        0.019273                 0.184615     0
5   89.220000            0.031393                 0.016368        0.013405                 0.199796     0
6   89.295000            0.084062                 0.081373        0.002401                 0.234579     0
7   89.480000            0.207178             

    ETF Price  ETF Change Price %  Net Asset Value Change%  Arbitrage in $  ETF Trading Spread in $  Flag
t                                                                                                        
0   89.816215                 NaN                      NaN             NaN                 0.010462     0
1   89.857816            0.046318                 0.046149        0.000152                 0.010513     0
2   89.875458            0.019633                 0.023179        0.003187                 0.010455     0
3   89.949347            0.082213                 0.078979        0.002909                 0.010372     0
4   90.027035            0.086368                 0.047339        0.035137                 0.010462   111
5   89.981383           -0.050709                -0.069414        0.016831                 0.010666   111
6   90.005813            0.027151                 0.006198        0.018859                 0.010817   111
7   89.966032           -0.044198             

KeyError: 17